<a href="https://colab.research.google.com/github/Farihaz2812/Cross-age-celebrity-dataset/blob/main/Cross_Age_Celebrity_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**CROSS AGE CELEBRITY DATASET**



Mounting Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Point to your dataset folder

In [ ]:
import os

# Path to dataset in Drive
DATA_DIR = "/content/drive/MyDrive/CACD"

# Check dataset
print("Number of files:", len(os.listdir(DATA_DIR)))
print("Sample files:", os.listdir(DATA_DIR)[:10])

Number of files: 2
Sample files: ['crop_part1', '.ipynb_checkpoints']


Parse filenames into a DataFrame

In [ ]:
import glob, pandas as pd

files = glob.glob(os.path.join(DATA_DIR, "crop_part1", "**/*"), recursive=True)
rows=[]

def parse_filename(f):
    fn = os.path.basename(f)
    try:
        age, gender, race = fn.split("_")[:3]
        return int(age), int(gender), int(race)
    except:
        return None

for f in files:
    result = parse_filename(f)
    if result:
        age, gender, race = result
        rows.append({"file": f, "age": age, "gender": gender, "race": race})

df = pd.DataFrame(rows)
print("Total parsed images:", len(df))
df.head()


Total parsed images: 4087


,file,age,gender,race
0,/content/drive/MyDrive/CACD/crop_part1/84_1_4_...,84,1,4
1,/content/drive/MyDrive/CACD/crop_part1/84_1_2_...,84,1,2
2,/content/drive/MyDrive/CACD/crop_part1/85_1_0_...,85,1,0
3,/content/drive/MyDrive/CACD/crop_part1/85_1_0_...,85,1,0
4,/content/drive/MyDrive/CACD/crop_part1/85_1_0_...,85,1,0


Split Train / Validation / Test

In [ ]:
from sklearn.model_selection import train_test_split

train_df, tmp = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(tmp, test_size=0.5, random_state=42)

print(len(train_df), len(val_df), len(test_df))


3269 409 409


Building tensorflow datasets

In [ ]:
import tensorflow as tf

IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image

def df_to_dataset(dataframe, label_col, batch_size=BATCH_SIZE, shuffle=True):
    paths = dataframe['file'].values
    labels = dataframe[label_col].values.astype('float32')
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    def load(path, label):
        img = preprocess_image(path)
        return img, label
    ds = ds.map(load, num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

# Example datasets
train_gender_ds = df_to_dataset(train_df, 'gender')
val_gender_ds = df_to_dataset(val_df, 'gender', shuffle=False)
test_gender_ds = df_to_dataset(test_df, 'gender', shuffle=False)

train_age_ds = df_to_dataset(train_df, 'age')
val_age_ds = df_to_dataset(val_df, 'age', shuffle=False)
test_age_ds = df_to_dataset(test_df, 'age', shuffle=False)


Data Augmentation


In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.08),
    tf.keras.layers.RandomZoom(0.06),
])


Gender Classification Model


In [ ]:
base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         include_top=False, weights='imagenet')
base.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_gender = tf.keras.Model(inputs, outputs)

model_gender.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

model_gender.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
#train:

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_gender.h5", save_best_only=True, monitor="val_accuracy"),
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True, monitor="val_accuracy")
]

history = model_gender.fit(train_gender_ds, validation_data=val_gender_ds, epochs=20, callbacks=callbacks)


Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5226 - loss: 0.7583

103/103 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.5226 - loss: 0.7581 - val_accuracy: 0.5599 - val_loss: 0.6884
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.5452 - loss: 0.7095 - val_accuracy: 0.5599 - val_loss: 0.6884
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.5441 - loss: 0.7107 - val_accuracy: 0.5599 - val_loss: 0.6846
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.5222 - loss: 0.7154 - val_accuracy: 0.5599 - val_loss: 0.6860
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.5474 - loss: 0.6994 - val_accuracy: 0.5599 - val_loss: 0.6843
Epoch 6/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5660 - loss: 0.6900

103/103 ━━━━━━━━━━━━━━━━━━━━ 271s 2s/step - accuracy: 0.5659 - loss: 0.6900 - val_accuracy: 0.5819 - val_loss: 0.6855
Epoch 7/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.5328 - loss: 0.6912 - val_accuracy: 0.5599 - val_loss: 0.6868
Epoch 8/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.5441 - loss: 0.6933 - val_accuracy: 0.5746 - val_loss: 0.6820
Epoch 9/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.5659 - loss: 0.6921 - val_accuracy: 0.5599 - val_loss: 0.6839
Epoch 10/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.5737 - loss: 0.6813 - val_accuracy: 0.5599 - val_loss: 0.6949
Epoch 11/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 271s 2s/step - accuracy: 0.5537 - loss: 0.6982 - val_accuracy: 0.5599 - val_loss: 0.7052
Epoch 12/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.5722 - loss: 0.6802 - val_accuracy: 0.5599 - val_loss: 0.6895


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

preds = (model_gender.predict(test_gender_ds) > 0.5).astype(int).squeeze()
y_true = test_df['gender'].values

print(classification_report(y_true, preds))
print(confusion_matrix(y_true, preds))


13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step
              precision    recall  f1-score   support

           0       0.71      0.09      0.16       187
           1       0.56      0.97      0.71       222

    accuracy                           0.57       409
   macro avg       0.63      0.53      0.43       409
weighted avg       0.63      0.57      0.46       409

[[ 17 170]
 [  7 215]]


Age Regression Model

In [ ]:
base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         include_top=False, weights='imagenet')
base.trainable = False

inp = tf.keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = data_augmentation(inp)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
out = tf.keras.layers.Dense(1, activation='linear')(x)

model_age = tf.keras.Model(inp, out)

model_age.compile(optimizer='adam', loss='mse',
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

model_age.summary()

callbacks = [tf.keras.callbacks.ModelCheckpoint("best_age.h5", save_best_only=True, monitor="val_loss"),
             tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)]

history = model_age.fit(train_age_ds, validation_data=val_age_ds, epochs=30, callbacks=callbacks)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_1 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_1 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

...

103/103 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - loss: 1429.1936 - mean_absolute_error: 37.2691 - val_loss: 1425.0461 - val_mean_absolute_error: 37.1713
Epoch 26/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1444.2179 - mean_absolute_error: 37.2825

103/103 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - loss: 1444.0240 - mean_absolute_error: 37.2803 - val_loss: 1424.1309 - val_mean_absolute_error: 37.4613
Epoch 27/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - loss: 1411.6027 - mean_absolute_error: 36.8237 - val_loss: 1439.2646 - val_mean_absolute_error: 37.6635
Epoch 28/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1443.2987 - mean_absolute_error: 37.4003

103/103 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - loss: 1443.1829 - mean_absolute_error: 37.3983 - val_loss: 1413.2914 - val_mean_absolute_error: 37.0682
Epoch 29/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1428.6002 - mean_absolute_error: 37.1631

103/103 ━━━━━━━━━━━━━━━━━━━━ 272s 2s/step - loss: 1428.6265 - mean_absolute_error: 37.1628 - val_loss: 1410.2720 - val_mean_absolute_error: 37.0003
Epoch 30/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1425.1466 - mean_absolute_error: 37.1182

103/103 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - loss: 1425.0707 - mean_absolute_error: 37.1172 - val_loss: 1406.3196 - val_mean_absolute_error: 37.1370


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

preds = model_age.predict(test_age_ds).squeeze()
y_true = test_df['age'].values

mae = mean_absolute_error(y_true, preds)
mse = mean_squared_error(y_true, preds)
rmse = np.sqrt(mse)
print("MAE:", mae, "RMSE:", rmse)

13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
MAE: 36.89067840576172 RMSE: 37.2466179253231


Preparing Dataset


In [ ]:
import os, glob, pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Path to dataset folder in Google Drive
DATA_DIR = "/content/drive/MyDrive/CACD"

files = glob.glob(os.path.join(DATA_DIR, "crop_part1", "**/*"), recursive=True)
rows=[]
for f in files:
    try:
        fn = os.path.basename(f)
        age, gender, race = fn.split("_")[:3]
        rows.append({"file": f, "age": int(age), "gender": int(gender), "race": int(race)})
    except:
        continue

df = pd.DataFrame(rows)
train_df, tmp = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(tmp, test_size=0.5, random_state=42)

print(len(train_df), len(val_df), len(test_df))

3269 409 409


Creating Data Pipeline


In [ ]:
IMG_SIZE = 128  # smaller for CNN from scratch
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image

def df_to_dataset(dataframe, label_col, batch_size=BATCH_SIZE, shuffle=True):
    paths = dataframe['file'].values
    labels = dataframe[label_col].values.astype('float32')
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    def load(path, label):
        img = preprocess_image(path)
        return img, label
    ds = ds.map(load, num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

# Example datasets
train_gender_ds = df_to_dataset(train_df, 'gender')
val_gender_ds   = df_to_dataset(val_df, 'gender', shuffle=False)
test_gender_ds  = df_to_dataset(test_df, 'gender', shuffle=False)

train_age_ds = df_to_dataset(train_df, 'age')
val_age_ds   = df_to_dataset(val_df, 'age', shuffle=False)
test_age_ds  = df_to_dataset(test_df, 'age', shuffle=False)


CNN for Gender Classification

In [ ]:
from tensorflow.keras import layers, models

def build_cnn_classifier(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')   # Binary classification
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

cnn_gender = build_cnn_classifier()
cnn_gender.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("cnn_gender.h5", save_best_only=True, monitor="val_accuracy"),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")
]

history_gender = cnn_gender.fit(train_gender_ds, validation_data=val_gender_ds,
                                epochs=20, callbacks=callbacks)


Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5549 - loss: 0.7665

103/103 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.5551 - loss: 0.7658 - val_accuracy: 0.6479 - val_loss: 0.6239
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6743 - loss: 0.6146

103/103 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.6745 - loss: 0.6143 - val_accuracy: 0.7066 - val_loss: 0.6171
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7055 - loss: 0.5838

103/103 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.7057 - loss: 0.5834 - val_accuracy: 0.7726 - val_loss: 0.4901
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7719 - loss: 0.4603

103/103 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.7719 - loss: 0.4603 - val_accuracy: 0.7897 - val_loss: 0.4348
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8205 - loss: 0.3971

...

103/103 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9164 - loss: 0.2043 - val_accuracy: 0.9095 - val_loss: 0.2653
Epoch 8/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.9503 - loss: 0.1468 - val_accuracy: 0.8900 - val_loss: 0.2511
Epoch 9/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9530 - loss: 0.1263

103/103 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9531 - loss: 0.1262 - val_accuracy: 0.9315 - val_loss: 0.1663
Epoch 10/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9809 - loss: 0.0604

103/103 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9809 - loss: 0.0604 - val_accuracy: 0.9682 - val_loss: 0.0818
Epoch 11/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9833 - loss: 0.0496

103/103 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.9834 - loss: 0.0495 - val_accuracy: 0.9804 - val_loss: 0.0534
Epoch 12/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.9947 - loss: 0.0244 - val_accuracy: 0.9682 - val_loss: 0.0761
Epoch 13/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.9889 - loss: 0.0366 - val_accuracy: 0.9804 - val_loss: 0.0617
Epoch 14/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.9827 - loss: 0.0500 - val_accuracy: 0.9804 - val_loss: 0.0672
Epoch 15/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9946 - loss: 0.0181

103/103 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.9945 - loss: 0.0183 - val_accuracy: 0.9829 - val_loss: 0.0634
Epoch 16/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.9958 - loss: 0.0191 - val_accuracy: 0.9780 - val_loss: 0.0490
Epoch 17/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.9953 - loss: 0.0157 - val_accuracy: 0.9804 - val_loss: 0.0667
Epoch 18/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9779 - loss: 0.0740 - val_accuracy: 0.9731 - val_loss: 0.0805
Epoch 19/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9883 - loss: 0.0311 - val_accuracy: 0.9829 - val_loss: 0.0795
Epoch 20/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.9923 - loss: 0.0206 - val_accuracy: 0.9780 - val_loss: 0.0742


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

preds = (cnn_gender.predict(test_gender_ds) > 0.5).astype(int).squeeze()
y_true = test_df['gender'].values

print(classification_report(y_true, preds))
print(confusion_matrix(y_true, preds))


13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 528ms/step
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       187
           1       0.96      0.99      0.97       222

    accuracy                           0.97       409
   macro avg       0.97      0.97      0.97       409
weighted avg       0.97      0.97      0.97       409

[[177  10]
 [  3 219]]


CNN for Age Regression

In [ ]:
def build_cnn_regressor(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='linear')   # Regression
    ])
    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])
    return model

cnn_age = build_cnn_regressor()
cnn_age.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("cnn_age.h5", save_best_only=True, monitor="val_loss"),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss")
]

history_age = cnn_age.fit(train_age_ds, validation_data=val_age_ds,
                          epochs=30, callbacks=callbacks)


Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 1837.3967 - mean_absolute_error: 39.2146

103/103 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - loss: 1834.6818 - mean_absolute_error: 39.1893 - val_loss: 1298.9552 - val_mean_absolute_error: 30.1949
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 899.7562 - mean_absolute_error: 24.7309

103/103 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 898.0389 - mean_absolute_error: 24.7009 - val_loss: 426.5154 - val_mean_absolute_error: 15.7678
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - loss: 568.1614 - mean_absolute_error: 18.5801 - val_loss: 461.6264 - val_mean_absolute_error: 15.5039
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 509.4454 - mean_absolute_error: 17.1649

103/103 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - loss: 508.6849 - mean_absolute_error: 17.1503 - val_loss: 318.4075 - val_mean_absolute_error: 14.2461
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 406.5163 - mean_absolute_error: 15.2406

103/103 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - loss: 406.2360 - mean_absolute_error: 15.2340 - val_loss: 268.6572 - val_mean_absolute_error: 11.7941
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 294.2374 - mean_absolute_error: 12.6612

...

103/103 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - loss: 85.1128 - mean_absolute_error: 6.3612 - val_loss: 75.8105 - val_mean_absolute_error: 5.6030
Epoch 22/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - loss: 69.8224 - mean_absolute_error: 6.0482 - val_loss: 80.9574 - val_mean_absolute_error: 5.7433
Epoch 23/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 63.5498 - mean_absolute_error: 5.5915

103/103 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 63.6159 - mean_absolute_error: 5.5944 - val_loss: 73.1614 - val_mean_absolute_error: 5.7341
Epoch 24/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 77.9253 - mean_absolute_error: 6.2961 - val_loss: 97.2050 - val_mean_absolute_error: 7.0579
Epoch 25/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 80.2127 - mean_absolute_error: 6.2602

103/103 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 80.1599 - mean_absolute_error: 6.2586 - val_loss: 67.0964 - val_mean_absolute_error: 5.0414
Epoch 26/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 59.7759 - mean_absolute_error: 5.3694 - val_loss: 98.8739 - val_mean_absolute_error: 6.7860
Epoch 27/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 62.8531 - mean_absolute_error: 5.5652

103/103 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - loss: 62.8396 - mean_absolute_error: 5.5643 - val_loss: 61.7404 - val_mean_absolute_error: 4.7151
Epoch 28/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 78.5661 - mean_absolute_error: 6.2990

103/103 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - loss: 78.5346 - mean_absolute_error: 6.2973 - val_loss: 57.8826 - val_mean_absolute_error: 4.3955
Epoch 29/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 71.2098 - mean_absolute_error: 5.8959 - val_loss: 75.6647 - val_mean_absolute_error: 5.5147
Epoch 30/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 62.8476 - mean_absolute_error: 5.6020 - val_loss: 59.2240 - val_mean_absolute_error: 4.5308


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

preds = cnn_age.predict(test_age_ds).squeeze()
y_true = test_df['age'].values

mae = mean_absolute_error(y_true, preds)
mse = mean_squared_error(y_true, preds)
rmse = np.sqrt(mse)
print("MAE:", mae, "RMSE:", rmse)

13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 490ms/step
MAE: 4.320807933807373 RMSE: 7.394510872677285
